## OpenWeatherMap API

OpenWeatherMap es un servicio en línea que proporciona datos meteorológicos globales a través de una API, incluyendo datos meteorológicos actuales, pronósticos, pronósticos inmediatos y datos históricos para cualquier ubicación geográfica.

**_Documentación_**: https://openweathermap.org/api

### Current

Es un servicio que retorna datos meteorológicos actuales de un punto usando latitud y longitud.

**_Documentación_**: https://openweathermap.org/current
```html
endpoint: https://api.openweathermap.org/data/2.5/weather
```

`Params:`

- **lat, lon** : _required_ : Geographical coordinates (latitude, longitude). If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around, please use our Geocoding API.


- **appid** : _required_ : Your unique API key (you can always find it on your account page under the "API key" tab).


- **mode** : _optional_ : Response format. Possible values are xml and html. If you don't use the mode parameter format is JSON by default.


- **units** :  _optional_ : Units of measurement. standard, **metric** and imperial units are available. If you do not use the units parameter, standard units will be applied by default.


- **lang** : _optional_ : You can use this parameter to get the output in your language.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib # Para ver la versión
import matplotlib.pyplot as plt
import seaborn as sns

import plotly # Para ver la versión
import plotly.express as px

import requests

from datetime import datetime
from time import sleep

from pprint import pprint

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"matplotlib=={matplotlib.__version__}")
print(f"sns=={sns.__version__}")
print(f"plotly=={plotly.__version__}")
print(f"requests=={requests.__version__}")

In [ ]:
api_key = "cd31574dc65a379909d4f921033a7e96"

lat, lon = 40.416775, -3.703790 # Madrid, Spain

endpoint = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

response = requests.get(url = endpoint)

data = response.json()

pprint(data)

In [ ]:
# Usando un diccionario como parámetros

params = {"appid" : api_key,
          "lat"   : lat,
          "lon"   : lon,
          "units" : "metric"}

endpoint = f"https://api.openweathermap.org/data/2.5/weather"

response = requests.get(url = endpoint, params = params)

data = response.json()

pprint(data, sort_dicts = False)

In [ ]:
response.url

In [ ]:
def get_current_weather(lat, lon, api_key):
    
    params = {"appid" : api_key,
              "lat"   : lat,
              "lon"   : lon,
              "units" : "metric"}

    endpoint = f"https://api.openweathermap.org/data/2.5/weather"

    response = requests.get(url = endpoint, params = params)

    return response.json()

In [ ]:
# Lat & Lon de capitales de España

df = pd.read_html(io = "https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales")

df = df[1]

# Lat & Lon
df["Lat ETRS89"] = df["Lat ETRS89"]/100_000_000
df["Lon ETRS89"] = df["Lon ETRS89"]/100_000_000

# Renombrar columnas
df.rename(mapper  = {col : col.split(" ")[0] for col in df.columns if "ETRS89" in col},
          axis    = 1,
          inplace = True)

weather_data = list()

for lat, lon in df[["Lat", "Lon"]].values:

    data = get_current_weather(lat = lat, lon = lon, api_key = api_key)

    try:
        description = data["weather"][0]["description"]
        temp        = data["main"]["temp"]
        feels_like  = data["main"]["feels_like"]
        temp_min    = data["main"]["temp_min"]
        temp_max    = data["main"]["temp_max"]
        pressure    = data["main"]["pressure"]
        humidity    = data["main"]["humidity"]
        wind_speed  = data["wind"]["speed"]
        dt          = data["dt"]
        lat         = data["coord"]["lat"]
        lon         = data["coord"]["lon"]
        name        = data["name"]

        weather_data.append([name, description, temp, feels_like, temp_min, 
                             temp_max, pressure, humidity, wind_speed, dt, lat, lon])
    except:
        pass

    sleep(0.1)

df_weather = pd.DataFrame(data   = weather_data,
                         columns = ["capital", "description", "temp", "feels_like", "temp_min",
                                    "temp_max", "pressure", "humidity", "wind_speed", "dt", "lat", "lon"])

df_weather

### Air Pollution (Historical)

_**Documentación:**_ https://openweathermap.org/api/air-pollution

```html
endpoint: http://api.openweathermap.org/data/2.5/air_pollution/history
```

Air Pollution API proporciona datos actuales, pronósticos e históricos sobre la contaminación del aire para cualquier coordenada.

Además del Índice de Calidad del Aire (Air Quality Index - **aqi**) básico, la API devuelve datos sobre gases contaminantes, como:

- Carbono monóxido (**CO**)
- Nitrogeno monóxido (**NO**)
- Dióxido de nitrógeno (**NO2**)
- Ozono (**O3**)
- Dióxido de azufre (**SO2**)
- Amoniaco (**NH3**)
- Partículas (**PM2.5** y **PM10**)


| Qualitative name | Index | SO2 (μg/m3)       | NO2 (μg/m3)       | PM10 (μg/m3)      | PM2.5 (μg/m3)     | O3 (μg/m3)        | CO (μg/m3)           |
|------------------|-------|------------|------------|------------|------------|------------|---------------|
| Good             | 1     | [0; 20)    | [0; 40)    | [0; 20)    | [0; 10)    | [0; 60)    | [0; 4400)     |
| Fair             | 2     | [20; 80)   | [40; 70)   | [20; 50)   | [10; 25)   | [60; 100)  | [4400; 9400)  |
| Moderate         | 3     | [80; 250)  | [70; 150)  | [50; 100)  | [25; 50)   | [100; 140) | [9400-12400)  |
| Poor             | 4     | [250; 350) | [150; 200) | [100; 200) | [50; 75)   | [140; 180) | [12400; 15400)|
| Very Poor        | 5     | ⩾350       | ⩾200       | ⩾200      | ⩾75        | ⩾180       | ⩾15400       |


`Params:`
- **lat, lon** : _required_ :	Geographical coordinates (latitude, longitude). If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around, please use our Geocoding API.


- **appid** : _required_ : Your unique API key (you can always find it on your account page under the "API key" tab)


- **start** : _optional_ : Start date (unix time, UTC time zone), e.g. start=1369728000


- **end** : _optional_ : End date (unix time, UTC time zone), e.g. end=1369789200

In [ ]:
start = int(datetime(2023, 1, 1).timestamp()) # 2023-01-01

end = int(datetime.now().timestamp()) # Hoy

print(f"start: {start}\nend: {end}")

In [ ]:
params = {"appid" : api_key,
          "lat"   : lat,
          "lon"   : lon,
          "start" : start,
          "end"   : end}

endpoint = "http://api.openweathermap.org/data/2.5/air_pollution/history"

response = requests.get(url = endpoint, params = params)

data = response.json()

pprint(data)

In [ ]:
df_air = pd.json_normalize(data["list"])

df_air

In [ ]:
# Renombramos las columnas

df_air.rename(mapper  = {x : x.split(".")[1] for x in df_air.columns[1:]},
              axis    = 1,
              inplace = True)

In [ ]:
df_air

In [ ]:
# Timestamp a datetime

df_air["dt"] = df_air["dt"].apply(lambda x : datetime.fromtimestamp(x))

df_air

In [ ]:
px.line(data_frame = df_air,
        x          = "dt",
        y          = "no2")

In [ ]:
df_air["day_of_year"] = df_air["dt"].apply(lambda x : x.strftime("%Y-%m-%d"))

df_air.head()

In [ ]:
df_air2 = df_air.groupby("day_of_year", as_index = False).agg({col : "mean" for col in df_air.columns[1:-1]})

df_air2

In [ ]:
# Line plot: media de valores de componentes por día

fig = px.line(data_frame = df_air2,
              x          = "day_of_year",
              y          = df_air2.columns[1:])

fig.show()

In [ ]:
df_air["hour"] = df_air["dt"].apply(lambda x : x.hour)

df_air

In [ ]:
df_air3 = df_air.groupby("hour", as_index = False).agg({col : "mean" for col in df_air.columns[1:-2]})

df_air3

In [ ]:
# Area plot: media de componentes agrupados por hora
fig = px.area(data_frame = df_air3,
              x          = "hour",
              y          = df_air3.columns[1:])

fig.show()

In [ ]:
# Boxplot

fig = px.box(data_frame = df_air,
             x          = "hour",
             y          = df_air.columns[2],
             hover_data = ["day_of_year"])

fig.show()

In [ ]:
################################################################################################################################